# Optimizing Engagement Opportunities for Posts on the Hacker News Community

The following project analyzes data from Hacker News posts between September 2015 and September 2016. Hacker News is a technology and entrepreneurship discussion forum for enthusiasts of various interests run by Y Combinator, a startup incubator.

My analysis aims to answer two essential questions. 

1. Which of `Ask HN` or `Show HN` posts gain more comments tranction or points traction on the discussion forum? 
2. What is the optimal time to post to gain the most comments or points for a post?

Note that `Ask HN` posts are posts when users ask the community a question, such as for feedback, technical help, or opinions. `Show HN` posts are those that show the Hacker News community anything the user has made or found interesting, such as a project. As such, finding data on what time of day is optimal to make a post will help any Hacker News users maximize the chance their `Ask HN` or `Show HN` post gains the most responses to their questions or points.

Because this project works by analyzing the number of comments, the original dataset, which may be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts), was reduced to around 20,000 rows of post data by removing any posts that did not have comments and randomly filtering the rest to this [dataset](https://dq-content.s3.amazonaws.com/356/hacker_news.csv).

I begin by importing in the modified dataset and removing the header.

In [28]:
import csv
opened_file = open('hacker_news.csv')
read_file = csv.reader(opened_file)
hn = list(read_file)

print(hn[0:3]) # first three rows, including the header
print('\n')

header = hn[0]
hn = hn[1:]

print(header)
print('\n')
print(hn[0:3]) # first three rows with the header row removed



[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']]


['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water

### Traction of Each Type of Post: Comments

I now extract `Ask HN` and `Show HN` posts by sorting them into their own lists to calculate how much traction each type of post gains, as defined by the average number of comments under each post type.

In [29]:
ask_hn_posts = []
show_hn_posts = []
other_posts = []

for row in hn:
    title = row[1].lower() # 1 is the index for post name
    if title.startswith('ask hn'):
        ask_hn_posts.append(row)
    elif title.startswith('show hn'):
        show_hn_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_hn_posts)) # 1744 posts
print(len(show_hn_posts)) # 1162 posts
print(len(other_posts)) # 17194 posts

1744
1162
17194


With the types of posts separated into their own lists, I can now loop through each to determine the average number of comments under each post in the `ask_hn_posts` and `show_hn_posts` lists.



In [30]:
total_ask_comments = 0
total_show_comments = 0
total_ask_posts = len(ask_hn_posts)
total_show_posts = len(show_hn_posts)

for row in ask_hn_posts:
    total_ask_comments += int(row[4]) # 4 is the index for number of comments
    
for row in show_hn_posts:
    total_show_comments += int(row[4])
    
avg_ask_comments = round(total_ask_comments / total_ask_posts, 2)
avg_show_comments = round(total_show_comments / total_show_posts, 2)

print(avg_ask_comments) # 14.0 average comments per ask post
print(avg_show_comments) # 10.3 average comments per show post
    

14.04
10.32


Ask posts tend to receive more comments than show posts, which makes sense given that ask posts are directly asking the community for answers, engagement, and feedback.

### Traction of Each Type of Post: Points

Using the lists in the previous step, I can now calculate the average number of points that each type of post brings in. On Hacker News, points is defined as the number of upvotes minus the number of downvotes on each post. This will help me determine which type of post, `Ask HN` or `Show HN`, receives more positive traction on average.

In [31]:
total_ask_points = 0
total_show_points = 0

for row in ask_hn_posts:
    total_ask_points += int(row[3]) # 3 is the index for number of points
    
for row in show_hn_posts:
    total_show_points += int(row[3])
    
avg_ask_points = round(total_ask_points / total_ask_posts, 2)
avg_show_points = round(total_show_points / total_show_posts, 2)

print(avg_ask_points) # 15.06 average points per ask post
print(avg_show_points) # 27.56 average points per show post

15.06
27.56


Interestingly, show posts receive significantly more points than ask posts do, which may be attributed to users being more inclined to simply upvote a post on something they agree with or find interesting rather than make a full comment.

Ask posts tend to receive more comments and show posts tend to receive more points. For Hacker News users who want to ensure their ask posts receive the most comments answering their questions and their show posts the most points to validate their project, it makes sense to then determine what time of day is best to make such a post.

As such, the following time analysis will be divided into two parts to answer the two questions:

1. What is the best time to create an ask post to receive the most comments?
2. What is the best time to create a show post to recieve the most points?

### Time Analysis: Ask Posts

I now determine the number of ask posts created in each hour of the day and how many comments are made in each hour group, starting by importing in the datetime module to make working with the `created_at` column of the dataset (index 6) easier. Note that the [dataset source](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts) lists hours in the `created_at` column in Eastern Time.

This analysis will help determine which hours see ask posts that receive the highest average number of comments.

In [5]:
import datetime as dt

ask_posts_list = []

# appends each post's created time and number of comments as a two-item list
for row in ask_hn_posts:
    ask_posts_list.append([row[6], int(row[4])]) 

ask_posts_by_hour = {}
ask_comments_by_hour = {}
date_format = '%m/%d/%Y %H:%M'

# generates frequency tables for the # of posts by hour and # of comments by hour 
for row in ask_posts_list:
    date = row[0]
    hour = dt.datetime.strptime(date, date_format).strftime('%H')
    comments = row[1]
    
    if hour in ask_posts_by_hour:
        ask_posts_by_hour[hour] += 1
        ask_comments_by_hour[hour] += comments
    else:
        ask_posts_by_hour[hour] = 1
        ask_comments_by_hour[hour] = comments

print('Number of posts by hour:')
print(ask_posts_by_hour)
print('\n')
print('Number of comments by hour:')
print(ask_comments_by_hour)

Number of posts by hour:
{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


Number of comments by hour:
{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


With these two dictionaries I can now determine the average number of comments per `Ask HN` post in each hour by dividing the key-value pairs of the two dictionaries by each other.

In [6]:
avg_comments_hour = []

for row in ask_comments_by_hour:
    average_comments = [row, round(ask_comments_by_hour[row] / ask_posts_by_hour[row], 2)]
    avg_comments_hour.append(average_comments)
    
print(avg_comments_hour)

[['09', 5.58], ['13', 14.74], ['10', 13.44], ['14', 13.23], ['16', 16.8], ['23', 7.99], ['12', 9.41], ['17', 11.46], ['15', 38.59], ['21', 16.01], ['20', 21.52], ['02', 23.81], ['18', 13.2], ['03', 7.8], ['05', 10.09], ['19', 10.8], ['01', 11.38], ['22', 6.75], ['08', 10.25], ['04', 7.17], ['00', 8.13], ['06', 9.02], ['07', 7.85], ['11', 11.05]]


This list can be formatted by descending average comments per hour to make it easier to identify which hours had the highest traction in terms of average comments on `Ask HN` posts.

In [12]:
swap_avg_comments_hour = []

# swaps the hour and comments columns so table can be sorted in desc. order by avg # of comments
for row in avg_comments_hour:
    swapped = [row[1], row[0]]
    swap_avg_comments_hour.append(swapped)
    
sorted_swap = sorted(swap_avg_comments_hour, reverse = True)

print(sorted_swap)
print('\n')

print('Top 5 Hours for Ask Posts Comments')

for avg, hour in sorted_swap[0:5]:
    hour_format = dt.datetime.strptime(hour, '%H').strftime('%H:%M')
    final_string = 'There are, on average, {:.2f} comments/post in hour {}'.format(avg, hour_format)
    print(final_string)
    

[[38.59, '15'], [23.81, '02'], [21.52, '20'], [16.8, '16'], [16.01, '21'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [13.2, '18'], [11.46, '17'], [11.38, '01'], [11.05, '11'], [10.8, '19'], [10.25, '08'], [10.09, '05'], [9.41, '12'], [9.02, '06'], [8.13, '00'], [7.99, '23'], [7.85, '07'], [7.8, '03'], [7.17, '04'], [6.75, '22'], [5.58, '09']]


Top 5 Hours for Ask Posts Comments
There are, on average, 38.59 comments/post in hour 15:00
There are, on average, 23.81 comments/post in hour 02:00
There are, on average, 21.52 comments/post in hour 20:00
There are, on average, 16.80 comments/post in hour 16:00
There are, on average, 16.01 comments/post in hour 21:00


The hour with by far the highest number of average comments per `Ask HN` post (out of posts that received comments) is 3:00 p.m. Eastern Time. The period from 8:00 p.m. to the end of 9:00 p.m. Eastern Time also tends to see a high number of comments per post. As such, Hacker News users may wish to post their questions during this time to maximize the odds of high engagement.

### Time Analysis: Show Posts

Show posts tend to receive more points than ask posts, a fact this analysis builds on to help determine which hours show posts tend to receive the highest average number of points.

In [22]:
show_posts_list = []

# appends each post's created time and number of points as a two-item list
for row in show_hn_posts:
    show_posts_list.append([row[6], int(row[3])]) 

show_posts_by_hour = {}
show_points_by_hour = {}
date_format = '%m/%d/%Y %H:%M'

# generates frequency tables for the # of posts by hour and # of points by hour
for row in show_posts_list:
    date = row[0]
    hour = dt.datetime.strptime(date, date_format).strftime('%H')
    points = row[1]
    
    if hour in show_posts_by_hour:
        show_posts_by_hour[hour] += 1
        show_points_by_hour[hour] += points
    else:
        show_posts_by_hour[hour] = 1
        show_points_by_hour[hour] = points

print('Number of posts by hour:')
print(show_posts_by_hour)
print('\n')
print('Number of points by hour:')
print(show_points_by_hour)

Number of posts by hour:
{'14': 86, '22': 46, '18': 61, '07': 26, '20': 60, '05': 19, '16': 93, '19': 55, '15': 78, '03': 27, '17': 93, '06': 16, '02': 30, '13': 99, '08': 34, '21': 47, '04': 26, '11': 44, '12': 61, '23': 36, '09': 30, '01': 28, '10': 36, '00': 31}


Number of points by hour:
{'14': 2187, '22': 1856, '18': 2215, '07': 494, '20': 1819, '05': 104, '16': 2634, '19': 1702, '15': 2228, '03': 679, '17': 2521, '06': 375, '02': 340, '13': 2438, '08': 519, '21': 866, '04': 386, '11': 1480, '12': 2543, '23': 1526, '09': 553, '01': 700, '10': 681, '00': 1173}


Similar to the analysis with ask posts, the pairs in the two dictionaries for show posts can be divided into each other to determine the average number of points per `Show HN` post in each hour.

In [23]:
avg_points_hour = []

for row in show_points_by_hour:
    average_points = [row, round(show_points_by_hour[row] / show_posts_by_hour[row], 2)]
    avg_points_hour.append(average_points)
    
print(avg_points_hour)


[['14', 25.43], ['22', 40.35], ['18', 36.31], ['07', 19.0], ['20', 30.32], ['05', 5.47], ['16', 28.32], ['19', 30.95], ['15', 28.56], ['03', 25.15], ['17', 27.11], ['06', 23.44], ['02', 11.33], ['13', 24.63], ['08', 15.26], ['21', 18.43], ['04', 14.85], ['11', 33.64], ['12', 41.69], ['23', 42.39], ['09', 18.43], ['01', 25.0], ['10', 18.92], ['00', 37.84]]


Like with the ask posts time analysis, the show posts list need to be sorted in descending order for readability.

In [24]:
swap_avg_points_hour = []

# swaps the hour and points columns so table can be sorted in desc. order by avg # of points
for row in avg_points_hour:
    swapped = [row[1], row[0]]
    swap_avg_points_hour.append(swapped)
    
sorted_swap = sorted(swap_avg_points_hour, reverse = True)

print(sorted_swap)
print('\n')

print('Top 5 Hours for Show Post Points')

for avg, hour in sorted_swap[0:5]:
    hour_format = dt.datetime.strptime(hour, '%H').strftime('%H:%M')
    final_string = 'There are, on average, {:.2f} points/post in hour {}'.format(avg, hour_format)
    print(final_string)
    

[[42.39, '23'], [41.69, '12'], [40.35, '22'], [37.84, '00'], [36.31, '18'], [33.64, '11'], [30.95, '19'], [30.32, '20'], [28.56, '15'], [28.32, '16'], [27.11, '17'], [25.43, '14'], [25.15, '03'], [25.0, '01'], [24.63, '13'], [23.44, '06'], [19.0, '07'], [18.92, '10'], [18.43, '21'], [18.43, '09'], [15.26, '08'], [14.85, '04'], [11.33, '02'], [5.47, '05']]


Top 5 Hours for Show Post Points
There are, on average, 42.39 points/post in hour 23:00
There are, on average, 41.69 points/post in hour 12:00
There are, on average, 40.35 points/post in hour 22:00
There are, on average, 37.84 points/post in hour 00:00
There are, on average, 36.31 points/post in hour 18:00


The top 5 hours with the most average points on `Show HN` posts seems to be much more spread out than the top 5 hours with most average comments on `Ask HN` posts. However, posts made during the evening on Eastern Time, including the stretch from roughly 10:00 p.m. to midnight, tend to see an average of more than 40 points/post.

Because the dataset includes only posts that received comments, I can only make the above conclusions for both the ask and show posts for the set of posts that received comments and not those with zero commenting engagement.

### Conclusion

In this project, I analyzed posts on the discussion site Hacker News based on their post type (`Ask HN` or `Show HN`), posting time, number of comments, and number of points. After determining that ask posts tended to average more comments and show posts tended to average more points, I determined the top 5 hours for which ask posts averaged the most comments and for which show posts averaged the most points.

I found that the mid-afternoon and mid-evening times resulted in the most comments on ask posts, while late-evening hours resulted in the most points on show posts. For Hacker News users who want to maximize the engagement on their posts, posting during these times will optimize the likelihood of getting the answers they need or the community responses they want.